<a href="https://colab.research.google.com/github/OVP2023/ML/blob/main/RS_dz1_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

1. Использовать датасет MovieLens, подготовка датасета для ML

In [ ]:
movies = pd.read_csv('/content/drive/MyDrive/Datasets/MovieLens/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Datasets/MovieLens/ratings.csv')
tags = pd.read_csv('/content/drive/MyDrive/Datasets/MovieLens/tags.csv')

In [ ]:
df=ratings
df=df.merge(movies,on='movieId',how='left')
df=df.merge(tags,on='movieId',how='left')
df.head(10)

,userId_x,movieId,rating,timestamp_x,title,genres,userId_y,tag,timestamp_y
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336.0,pixar,1.139046e+09
1,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474.0,pixar,1.137207e+09
2,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567.0,fun,1.525286e+09
3,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,289.0,moldy,1.143425e+09
4,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,289.0,old,1.143425e+09
5,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,NaN,NaN,NaN
6,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,424.0,mystery,1.457842e+09
7,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,424.0,twist ending,1.457842e+09
8,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,474.0,serial killer,1.137206e+09
9,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,424.0,mindfuck,1.457842e+09


In [ ]:
df['userId_mean_rating'] = ratings.groupby('userId')['rating'].mean()
df['userId_median_rating'] = ratings.groupby('userId')['rating'].median()
df['movieId_median_rating'] = ratings.groupby('movieId')['rating'].median()
df['movieId_mean_rating'] = ratings.groupby('movieId')['rating'].mean()
#X = df.dropna()
X=df.drop(columns=['movieId','userId_x','timestamp_x','userId_y','timestamp_y','title'])
X.head()

,rating,genres,tag,userId_mean_rating,userId_median_rating,movieId_median_rating,movieId_mean_rating
0,4.0,Adventure|Animation|Children|Comedy|Fantasy,pixar,NaN,NaN,NaN,NaN
1,4.0,Adventure|Animation|Children|Comedy|Fantasy,pixar,4.366379,5.0,4.0,3.920930
2,4.0,Adventure|Animation|Children|Comedy|Fantasy,fun,3.948276,4.0,3.5,3.431818
3,4.0,Comedy|Romance,moldy,2.435897,0.5,3.0,3.259615
4,4.0,Comedy|Romance,old,3.555556,4.0,3.0,2.357143


In [ ]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))


genres_list = X.genres.apply(change_string).tolist()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(genres_list)
X_train_counts.todense()[0].shape

(1, 20)

In [ ]:
count_vect.vocabulary_

{'adventure': 1,
 'animation': 2,
 'children': 3,
 'comedy': 4,
 'fantasy': 8,
 'romance': 15,
 'action': 0,
 'crime': 5,
 'thriller': 17,
 'mystery': 13,
 'horror': 10,
 'drama': 7,
 'war': 18,
 'western': 19,
 'scifi': 16,
 'musical': 12,
 'filmnoir': 9,
 'imax': 11,
 'documentary': 6,
 'nogenreslisted': 14}

In [ ]:
X_train_counts.todense()

matrix([[0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 1, 0, 0]])

In [ ]:
list_genres=['adventure','animation','children','comedy','fantasy','romance','action','crime','thriller','mystery','horror','drama','war','western','scifi','musical','filmnoir','imax','documentary','nogenreslisted']
df1=pd.DataFrame(X_train_counts.todense(),columns=list_genres)
df1.head()

,adventure,animation,children,comedy,fantasy,romance,action,crime,thriller,mystery,horror,drama,war,western,scifi,musical,filmnoir,imax,documentary,nogenreslisted
0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
df1['userId_mean_rating']=X['userId_mean_rating']
df1['userId_median_rating']=X['userId_median_rating']
df1['movieId_median_rating']=X['movieId_median_rating']
df1['movieId_mean_rating']=X['movieId_mean_rating']
df1['rating']=X['rating']
df1 = df1.dropna()
y=df1['rating']
df1=df1.drop(columns=['rating'])
df1.head()

,adventure,animation,children,comedy,fantasy,romance,action,crime,thriller,mystery,...,scifi,musical,filmnoir,imax,documentary,nogenreslisted,userId_mean_rating,userId_median_rating,movieId_median_rating,movieId_mean_rating
1,0,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,4.366379,5.0,4.0,3.920930
2,0,1,1,1,1,0,0,0,1,0,...,0,0,0,0,0,0,3.948276,4.0,3.5,3.431818
3,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,2.435897,0.5,3.0,3.259615
4,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,3.555556,4.0,3.0,2.357143
5,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,3.636364,4.0,3.0,3.071429


In [ ]:
from sklearn.preprocessing import normalize

normalize(df1, norm="l1")

array([[0.        , 0.04486858, 0.04486858, ..., 0.22434292, 0.17947433,
        0.17592658],
       [0.        , 0.05030157, 0.05030157, ..., 0.20120629, 0.17605551,
        0.17262585],
       [0.        , 0.        , 0.        , ..., 0.04466075, 0.2679645 ,
        0.29115374],
       ...,
       [0.05479393, 0.        , 0.        , ..., 0.16438178, 0.21917571,
        0.22553301],
       [0.05604847, 0.        , 0.        , ..., 0.16814542, 0.2241939 ,
        0.20017312],
       [0.05337885, 0.        , 0.        , ..., 0.18682599, 0.21351541,
        0.1892523 ]])

2.Построить рекомендации (регрессия, предсказываем оценку) на фичах:
    TF-IDF на тегах и жанрах;
    средние оценки (+ median, variance и т. д.) пользователя и фильма.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor  #Модель дерево решений регрессия


X_train, X_test, y_train, y_test = train_test_split(df1, y, test_size=0.2, random_state=0)

clf = DecisionTreeRegressor(min_samples_leaf=14, max_depth=3, random_state=10)
clf.fit(X_train, y_train)

print('Accuracy->',round(clf.score(X_test,y_test),3))

Accuracy-> 0.681


3.Оценить RMSE на тестовой выборке.

In [ ]:
from sklearn.metrics import root_mean_squared_error # импортируем функцию корня среднеквадратической ошибки

y_pred = clf.predict(X_test)
print('Root Mean Squared Error (RMSE):', root_mean_squared_error(y_test, y_pred))

Root Mean Squared Error (RMSE): 0.544308894377163
